# GbT μLM
Generative _BARELY_ Pretrained Transformer _MICRO_ Language Model

Bu seferki makalemiz [bu](https://arxiv.org/abs/1706.03762). Yaptığımız şeyleri bu safhada bu modelin en basit halini uygulayabilmek adına yaptık:

![Transformer](.assets/i8.png)

Modelin `encoder` yapısını gözardı edeceğiz, `decoder` içinde ise `self-attention` yapısına odaklanıyoruz (yani `cross-attension` mevhumundan bahsetmiyoruz):

![Attention](.assets/i9.png)

## "Scaled Dot-Product Attention" Nedir?

Çalışmamız kapsamında bu yapı ne iş görüyor (sadece dikkat mekanizması açısından önemli adımlara) adım adım bakalım:

1. Harfler (çalışmamızdaki _token_) için bir _embedding_ oluştur.
1. İstenen bağlam boyutu kadar (çalışmamızda en uzun kelime boyu +1) pozisyonlar için bir _embedding_ oluştur.
1. Bu iki _embedding_ değerini topla ve _girdiyi_ oluştur.
1. Bu girdi ile 3 doğrusal katman oluştur:
   - **q**: sorgu; iki harf benzer sorgulara sahipse birbirlerini ilgilendirirler
   - **k**: anahtar; her harfin kendini tanıtan bilgi
   - **v**: değer; her harfin bir diğerine aktaracağı bilgi. Sorgu ve anahtar bilgilerinin çarpımı o değerden ne kadar faydalanacağını belirliyor.
1. `q x transpose(k)` hesapla, normalize et.
1. Maske uygula (ve sonra normalize et): bağlam içindeki tüm harfler birbirini görürse "bir sonraki harfi tahmin etme" görevi anlamsız olacak.
1. Sonucu `v` ile çarparak _çıktıyı_ elde et.

Elde ettiğimiz çıktı, her harfin kendinden önceki harflerle kurduğu bağ üzerinden elde ettiği bir bilgi. Bu bilgi, şimdiye kadar kullandığımız "bağlamda şu 3-5 harf varsa bir sonraki harf şu olur" yaklaşımından çok daha güçlü: mesela model adını koyamasa da ünlü uyumu kurallarını keşfedebilir durumda. Bu bilgiyi elde ettikten sonra yeni maceralar peşinde koşmuyoruz aslında; bunu eldeki araçlar vasıtasıyla kullanıp daha tutarlı "uyduran" bir modele kavuşuyoruz.

Tabi elde ettiğimiz bu bilgi ile daha karmaşık yapılardan vazgeçebilir hale geliyoruz, dolayısıyla modelin eğitilebilirliği (zaman maliyeti açısından) kolaylaşıyor. Bu arada işlemlerin GPU üzerinde paralel olarak çalıştırılabilmesi de bu hızlı eğitimin önünü açan diğer kritik bileşen. Bunlar olmasaydı bugün transformer tarihteki vasat tercüme makalelerinden birinde geçen bir kavram olacaktı sadece.

En basit hali bir işe yaramasa da kod olarak burada bir dursun:

```py
BLOK, EMBED = 31, 60 # bunlar modelin "hiper" parametreleri
maske = torch.tril(torch.ones(BLOK, BLOK))
girdi = torch.randn(BLOK, EMBED) # bu dışarıdan gelecek (x), çalışsın diye öylesine bir değer verdik

# aslında bu katmanları bir kez oluşturup forward pass içinde mevcutları kullanmamız lazım;
# saçma olsa da ne olduğunu görmek adına böyle:
q = nn.Linear(EMBED, EMBED)(girdi)
k = nn.Linear(EMBED, EMBED)(girdi)
v = nn.Linear(EMBED, EMBED)(girdi)

carpim = q @ k.transpose(-2, -1)
maskeli = carpim.masked_fill(maske == 0, float('-inf'))

dikkat = F.softmax(maskeli, dim=-1)

cikti = dikkat @ v # (y)
```

## Hazırlık

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import math
from dataclasses import dataclass

alfabe = list('.abcçdefgğhıijklmnoöprsştuüvyz')
harf2idx = { harf:idx for idx, harf in enumerate(alfabe) }
idx2harf = { idx:harf for harf, idx in harf2idx.items() }

isimler = open("./isimler.txt", "r").read().splitlines()
maxUzunluk = max([len(isim) for isim in isimler])

def isle(isim):
  enc, ln = torch.tensor([harf2idx[h] for h in isim], dtype=torch.long), len(isim)
  x, y = torch.zeros(maxUzunluk + 1, dtype=torch.long), torch.zeros(maxUzunluk + 1, dtype=torch.long)
  x[1:1+ln] = enc
  y[:ln] = enc
  y[ln+1:] = -1
  return x, y

def xyOlustur(isimler):
  X, Y = [], []
  for isim in isimler:
    x, y = isle(isim)
    X.append(x) ; Y.append(y)
  X, Y = torch.stack(X), torch.stack(Y)
  print(X.shape, Y.shape)
  return X, Y

@torch.inference_mode()
@torch.no_grad()
def ornekUret(model, adet):
  model.eval()
  n_blk, ornekler = model.block_size, torch.zeros(adet, 1, dtype=torch.long)
  def kirp(x):
    y = x[1:].tolist() ; y = y[:y.index(0) if 0 in y else len(y)] ; return y # 0..0 arası lazım sadece, sonda 0 yoksa hepsi gelecek
  def metneCevir(x):
    return ''.join(idx2harf[h] for h in x)
  for _ in range(n_blk - 1):
    logits, _ = model(ornekler if ornekler.size(1) <= n_blk else ornekler[:, -n_blk:])
    siradaki = torch.multinomial(F.softmax(logits[:, -1, :], dim = -1), num_samples = 1)
    ornekler = torch.cat((ornekler, siradaki), dim = 1)
  model.train()
  return map(lambda x: metneCevir(kirp(x)), ornekler)

@torch.inference_mode()
@torch.no_grad()
def lossHesapla(model, X, Y):
  model.eval()
  _, loss = model(X, Y)
  model.train()
  return loss

n_alfabe = len(alfabe)
n_isim = len(isimler)

n_isim, n_alfabe, maxUzunluk, isle('abacılar')

(29996,
 30,
 30,
 (tensor([ 0,  1,  2,  1,  3, 11, 15,  1, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]),
  tensor([ 1,  2,  1,  3, 11, 15,  1, 21,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1,
          -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])))

## Veri Kümeleri

In [2]:
tumX, tumY = xyOlustur(isimler)
random.seed(5) ; karisik = random.sample(isimler, k = int(0.8 * n_isim))
trnX, trnY = xyOlustur(karisik)

torch.Size([29996, 31]) torch.Size([29996, 31])
torch.Size([23996, 31]) torch.Size([23996, 31])


## Soyutlamalar

In [3]:
# -----------------------------------------------------------------------------------------------
class MultiHeadSelfAttention(nn.Module):

  def __init__(self, config):
    super().__init__() ; assert config.n_embd % config.n_head == 0
    self.qkv = nn.Linear(config.n_embd, 3 * config.n_embd)
    self.c_proj = nn.Linear(config.n_embd, config.n_embd)
    self.register_buffer("bias", torch.tril(torch.ones(1, 1, config.block_size, config.block_size)))
    self.n_head = config.n_head
    self.n_embd = config.n_embd

  def forward(self, x):
    B, T, C = x.size() # BATCH, TIME = BLOCK, CHANNELS = N_EMBD

    q, k ,v  = self.qkv(x).split(self.n_embd, dim=2)
    k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
    v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

    att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
    att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
    att = F.softmax(att, dim=-1) # (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)

    y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
    y = self.c_proj(y.transpose(1, 2).contiguous().view(B, T, C))
    return y # (B, T, C)

# -----------------------------------------------------------------------------------------------
class Block(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.ln_1 = nn.LayerNorm(config.n_embd)
    self.attn = MultiHeadSelfAttention(config)
    self.ln_2 = nn.LayerNorm(config.n_embd)
    self.mlp = nn.ModuleDict(dict(
      c_fc  = nn.Linear(config.n_embd, 4 * config.n_embd),
      c_proj  = nn.Linear(4 * config.n_embd, config.n_embd),
      act   = nn.GELU(),
    ))
    m = self.mlp
    self.mlpf = lambda x: m.c_proj(m.act(m.c_fc(x))) # MLP forward

  def forward(self, x):
    x = x + self.attn(self.ln_1(x))
    x = x + self.mlpf(self.ln_2(x))
    return x

# -----------------------------------------------------------------------------------------------
class Transformer(nn.Module):

  def __init__(self, config):
    super().__init__()
    self.block_size = config.block_size

    self.transformer = nn.ModuleDict(dict(
      wte = nn.Embedding(config.vocab_size, config.n_embd),
      wpe = nn.Embedding(config.block_size, config.n_embd),
      h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
      ln_f = nn.LayerNorm(config.n_embd),
    ))
    self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

  def forward(self, idx, targets=None):
    _, T = idx.size() ; assert T <= self.block_size

    # x = token embeddings (B, T, C) + position embeddings (1, T, C)
    x = self.transformer.wte(idx) + self.transformer.wpe(torch.arange(0, T, dtype=torch.long).unsqueeze(0))
    for block in self.transformer.h:
      x = block(x)
    x = self.transformer.ln_f(x)
    logits = self.lm_head(x)
    loss = None if targets is None else F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)

    return logits, loss

## Model

In [4]:
torch.manual_seed(5)

@dataclass
class ModelConfig:
  block_size: int = maxUzunluk + 1
  vocab_size: int = n_alfabe
  n_layer: int = 4
  n_head: int = 4
  n_embd: int = n_head * 16

cfg = ModelConfig()
model = Transformer(cfg)
print(sum(p.numel() for p in model.parameters()))

205888


## Eğitim

In [5]:
torch.manual_seed(5)

optimizer = torch.optim.AdamW(model.parameters(), lr = 5e-4, weight_decay = 0.01, betas = (0.9, 0.99), eps = 1e-8)
enIyiDurum, enIyiLoss, enIyi = None, None, 0

for i in range(4000):
  bat = torch.randint(0, trnX.shape[0], (16,))
  _, loss = model(trnX[bat], trnY[bat])
  model.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()
  if enIyiLoss is None or loss < enIyiLoss:
    enIyiLoss, enIyiDurum, enIyi = loss, model.state_dict(), i
  if i % 1000 == 0: 
    print(loss.item())

model.load_state_dict(enIyiDurum)
print('-'*20)
print(f'trn: {enIyiLoss.item()} ({enIyi})')
print(f'tum: {lossHesapla(model, tumX, tumY).item()}')

3.601732015609741
2.0923452377319336
1.914368987083435
1.914691686630249
--------------------
trn: 1.5682162046432495 (3513)
tum: 1.8509806394577026


## Sonuç

In [6]:
torch.manual_seed(5)

inTrn, inTum, yeni = [], [], []
for ornek in ornekUret(model, 50):
  if ornek in karisik:
    inTrn.append(ornek)
  elif ornek in isimler:
    inTum.append(ornek)
  else:
    yeni.append(ornek)

for lst, yer in [(inTrn, 'eğitimde görüldü'), (inTum, 'eğitimde görülmedi'), (yeni, 'yeni')]:
  if len(lst) == 0: continue
  print('-*'*15)
  print(f"{len(lst)} örnek {yer}:")
  print('*-'*15)
  for ornek in lst:
    print(ornek)

-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
3 örnek eğitimde görüldü:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
karandere
camili
yaylıca
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
1 örnek eğitimde görülmedi:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
celi
-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*
46 örnek yeni:
*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-
karaeşi
çağşık
belersanlık
akşayamba
karadımğırı
tokurlar
çatırekki
kayupkırganı
elbeyoğlu
çivendiğ
datalık
çayhallı
şahrif
karımalan
kouluderesi
taltar
çanakdırca
yaycıgöze
ekingüney
kahaçten
mizileliler
mediroğlu
etekuyu
köpret
kalmaklı
bozuri
dakılıkışcınlı
bekdivançır
yivriserişlen
yamaserli
korumuşağı
soğarbayır
muslurhalli
havumler
elmaydalak
ekvanut
sevece
şicfiroğlu
düngimbi
elaköy
erekay
karabedi
ebsertepe
yukarıaç
balüyereci
çörükyus


In [ ]:
# modelin en iyi durumunu kaydettik, incelemek için aşağıdaki satırı açıp çalıştırmak yeterli
# enIyiDurum